
### Завдання 1: Виклик LLM з базовим промптом

Створіть можливість викликати LLM зі звичайним текстовим промптом.

Промпт має дозвляти отримати інформацію простою мовою на певну тему. В цьому завданні ми хочемо дізнатись про тему "Квантові обчислення".

Відповідь моделі повинна містити визначення, ключові переваги та поточні дослідження в цій галузі.

Обмежте відповідь до 200 символів і пропишіть в промпті аби відповідь була короткою (це зекономить Вам час і гроші на згенеровані токени).

В якості LLM можна скористатись як моделлю з HugginFace (рекомендую Mistral), так і ChatGPT4 або ChatGPT3. В обох випадках треба імпортувати потрібну "обгортку" (тобто клас, який дозволить ініціювати модель) з LangChain для виклику LLM за API, а також зчитати особистий токен з файла, наприклад, `creds.json`, який розміщений у Вас локально і Ви НЕ здаєте його в ДЗ і НЕ комітите в git 😏

Встановіть своє значення температури на свій розсуд (тут немає правильного чи неправильного значення) і напишіть, чому ви обрали саме таке значення для цього завдання.  

Запити можна робити як українською, так і англійською - орієнтуйтесь на те, де і чи хочете ви потім лишити цей проєкт і відповідна яка мова буде пасувати більше. В розвʼязках промпти - українською.

In [17]:
import json
import os

from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains.llm_math.base import LLMMathChain
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain_community.utilities import SerpAPIWrapper
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_huggingface import HuggingFaceEndpoint

from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch

In [22]:
# Read credentials from a local file
with open('../creds.json') as file:
  creds = json.load(file)

# Prepare the environment
os.environ["HUGGINGFACEHUB_API_TOKEN"] = creds['HUGGINGFACEHUB_API_TOKEN']
os.environ["OPENAI_API_KEY"] = creds["OPENAI_API_KEY"]
os.environ["SERPAPI_API_KEY"] = creds["SERPAPI_API_KEY"]
os.environ["TAVILY_API_KEY"] = creds["TAVILY_API_KEY"]
os.environ["OPENWEATHERMAP_API_KEY"] = creds['OPENWEATHERMAP_API_KEY']

In [3]:
# Set overall temperature. Set it between creativity and accuracy.
# Since the task is not about creative writing so it should not be more than ~0.5-0.6
overall_temperature = 0.2

mistral = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    temperature=overall_temperature,
    max_new_tokens=512,  # in general 1 English word = 1.3 tokens or 0.75 work costs 1 token :)
)

In [4]:
topic = "Квантові обчислення"

prompt = (
f"""
Ти розумний асистент, який пояснює складні теми простими словами. Твоя задача в максимально коротку відповідь надати визначення, ключові переваги та поточні дослідження в галузі {topic}.

Відповідь має бути короткою, не більше 200 символів.
Відповідаю тією мовою, якою запитують.
"""
)


In [6]:
# Call the model
# Invoke the model
response = mistral.invoke(prompt)

# Output the result
print(response)

/Users/dmytro/Code/MachineLearning/machine-learning-for-humans-2.0/.env/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/mistralai/Mistral-7B-Instruct-v0.3 (Request ID: Root=1-67f2879e-64c453921226729d0ceadfe8;bfd9ba29-498e-405f-be0f-cdfc01725ff2)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

### Нотатки для куратора

До цього все нормально запускалось, видимо, що там вийшов за ліміти.

### Завдання 2: Створення параметризованого промпта для генерації тексту
Тепер ми хочемо оновити попередній фукнціонал так, аби в промпт ми могли передавати тему як параметр. Для цього скористайтесь `PromptTemplate` з `langchain` і реалізуйте параметризований промпт та виклик моделі з ним.

Запустіть оновлений функціонал (промпт + модел) для пояснень про теми
- "Баєсівські методи в машинному навчанні"
- "Трансформери в машинному навчанні"
- "Explainable AI"

Виведіть результати відпрацювання моделі на екран.

In [7]:
template = (
"""Ти розумний асистент, який пояснює складні теми простими словами. Твоя задача в максимально коротку відповідь надати визначення, ключові переваги та поточні дослідження в галузі {topic}.

Відповідь має бути короткою, не більше 200 символів.
Відповідаю тією мовою, якою запитують.
"""
)

prompt = PromptTemplate.from_template(template)

In [8]:
# Call the model
# Invoke the model
llm_chain = prompt | mistral

topics = ["Баєсівські методи в машинному навчанні", "Трансформери в машинному навчанні", "Explainable AI"]

for topic in topics:
  print(llm_chain.invoke({'topic': topic}))

/Users/dmytro/Code/MachineLearning/machine-learning-for-humans-2.0/.env/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/mistralai/Mistral-7B-Instruct-v0.3 (Request ID: Root=1-67f287f3-574f0e963297a30c4af4315e;b8fe663c-cdd1-4770-8c42-8f4df0d0040f)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.



### Завдання 3: Використання агента для автоматизації процесів
Створіть агента, який допоможе автоматично шукати інформацію про останні наукові публікації в різних галузях. Наприклад, агент має знайти 5 останніх публікацій на тему штучного інтелекту.

**Кроки:**
1. Налаштуйте агента типу ReAct в LangChain для виконання автоматичних запитів.
2. Створіть промпт, який спрямовує агента шукати інформацію в інтернеті або в базах даних наукових публікацій.
3. Агент повинен видати список публікацій, кожна з яких містить назву, авторів і короткий опис.

Для взаємодії з пошуком там необхідно створити `Tool`. В лекції ми використовували `serpapi`. Можна продовжити користуватись ним, або обрати інше АРІ для пошуку (вони в тому числі є безкоштовні). Перелік різних АРІ, доступних в langchain, і орієнтир по вартості запитів можна знайти в окремому документі [тут](https://hannapylieva.notion.site/API-12994835849480a69b2adf2b8441cbb3?pvs=4).

Лишаю також нижче приклад використання одного з безкоштовних пошукових АРІ - DuckDuckGo (не потребує створення токена!)  - можливо він вам сподобається :)


In [8]:
#!pip install -q langchain_community duckduckgo_search

In [9]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("Obama's first name?")

"The White House, official residence of the president of the United States, in 2008. The president of the United States is the head of state and head of government of the United States, [1] indirectly elected to a four-year term via the Electoral College. [2] Under the U.S. Constitution, the officeholder leads the executive branch of the federal government and is the commander-in-chief of the ... Obama's father, Barack Obama, Sr., was a teenage goatherd in rural Kenya, won a scholarship to study in the United States, and eventually became a senior economist in the Kenyan government.Obama's mother, S. Ann Dunham, grew up in Kansas, Texas, and Washington state before her family settled in Honolulu.In 1960 she and Barack Sr. met in a Russian language class at the University of Hawaii ... Since 1789 and the election of George Washington, America's first president, 45 individuals have served as the chief executive of the United States ... Barack Obama: Joseph Biden: Democratic: 2009-2017: D

In [10]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=overall_temperature, max_tokens=512)

tools = load_tools(["serpapi"], llm=llm)

prompt = hub.pull("hwchase17/react")
react_prompt = prompt + "Always follow the ReAct format strictly. You're an extraction specialist."

/Users/dmytro/Code/MachineLearning/machine-learning-for-humans-2.0/.env/lib/python3.12/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [11]:
tavily_tool = TavilySearch(
    max_results=1,
    topic="news",
    include_answer=True
)

tools = [tavily_tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=6, )

In [12]:
# query = """Your task is to search for and identify the 5 most recent publications on this topic. For each publication, you need to provide the following information:
# 1. Title
# 2. Authors
# 3. A short description (1-2 sentences summarizing the main points of the publication)
# """

query = """Here is the topic you need to search for - Artificial Intelligence.

Your task is to search for and identify the 5 most recent publications on this topic. For each publication, you need to provide the following information:
1. Title
2. Authors
3. A short description (1-2 sentences summarizing the main points of the publication)

Please follow these steps:

1. Search for publications related to the given topic.
2. Identify the 5 most recent publications.
3. For each publication, extract the required information (title, authors, short description).
4. Format the results in a numbered list for easy readability.

Here's an example of how your final output should be formatted:

1. [Title of Publication 1]
   Authors: [Author 1], [Author 2], et al.
   Description: [1-2 sentence summary of the publication]

2. [Title of Publication 2]
   Authors: [Author 1], [Author 2], et al.
   Description: [1-2 sentence summary of the publication]

(Continue for all 5 publications)

Please ensure that your list is clear, concise, and easy to read. Remember to focus on the most recent publications and provide accurate information for each entry."""

agent_executor.invoke({'input': query })



> Entering new AgentExecutor chain...
I need to search for the most recent publications on Artificial Intelligence to gather the required information. 

Action: tavily_search  
Action Input: "recent publications on Artificial Intelligence 2023"  {'query': 'recent publications on Artificial Intelligence 2023', 'follow_up_questions': None, 'answer': 'Recent advancements in artificial intelligence were published in 2023, featuring research from the ICRAAI conference in Jaipur, India. The book includes papers presented at the event held in December 2023. This publication remains relevant for current AI developments.', 'images': [], 'results': [{'title': 'Recent Advancements in Artificial Intelligence - Springer', 'url': 'https://link.springer.com/book/10.1007/978-981-97-1111-6', 'content': 'Presents recent research in artificial intelligence; Discusses the outcomes of ICRAAI 2023, held in Jaipur, India ... This book features research papers presented at the Second International Conferenc

{'input': "Here is the topic you need to search for - Artificial Intelligence.\n\nYour task is to search for and identify the 5 most recent publications on this topic. For each publication, you need to provide the following information:\n1. Title\n2. Authors\n3. A short description (1-2 sentences summarizing the main points of the publication)\n\nPlease follow these steps:\n\n1. Search for publications related to the given topic.\n2. Identify the 5 most recent publications.\n3. For each publication, extract the required information (title, authors, short description).\n4. Format the results in a numbered list for easy readability.\n\nHere's an example of how your final output should be formatted:\n\n1. [Title of Publication 1]\n   Authors: [Author 1], [Author 2], et al.\n   Description: [1-2 sentence summary of the publication]\n\n2. [Title of Publication 2]\n   Authors: [Author 1], [Author 2], et al.\n   Description: [1-2 sentence summary of the publication]\n\n(Continue for all 5 pub



### Завдання 4: Створення агента-помічника для вирішення бізнес-задач

Створіть агента, який допомагає вирішувати задачі бізнес-аналітики. Агент має допомогти користувачу створити прогноз по продажам на наступний рік враховуючи рівень інфляції і погодні умови. Агент має вміти використовувати Python і ходити в інтернет аби отримати актуальні дані.

**Кроки:**
1. Налаштуйте агента, який працюватиме з аналітичними даними, заданими текстом. Користувач пише

```
Ми експортуємо апельсини з Бразилії. В 2021 експортували 200т, в 2022 - 190т, в 2023 - 210т, в 2024 який ще не закінчився - 220т. Зроби оцінку скільки ми зможемо експортувати апельсинів в 2025 враховуючи погодні умови в Бразилії і попит на апельсини в світі виходячи з економічної ситуації.
```

2. Створіть запит до агента, що містить чітке завдання – видати результат бізнес аналізу або написати, що він не може цього зробити і запит користувача (просто може бути все одним повідомлленням).

3. Запустіть агента і проаналізуйте результати. Що можна покращити?


In [20]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, max_tokens=512)

# Prepare the tools
python_repl = PythonREPL()
python_interpreter_tool = Tool(
    name="python_interpreter",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you necessarily should print it out with `print(...)`. Otherwise you won't see the result! It's very important.",
    func=python_repl.run,
)

problem_chain = LLMMathChain.from_llm(llm=llm)
math_tool = Tool.from_function(name="Calculator",
                func=problem_chain.run,
                description="Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions.")

tools = load_tools(['serpapi', 'openweathermap-api'], llm=llm)

tools.append(python_interpreter_tool)
tools.append(math_tool)

In [23]:
prompt = hub.pull('hwchase17/react')
react_prompt = prompt + "Always follow the ReAct format strictly. You're an extraction specialist."


agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=10)

query = """
Ми експортуємо апельсини з Бразилії. В 2021 — 200т, в 2022 — 190т, в 2023 — 210т, в 2024 (ще не завершено) — 220т.
Зроби оцінку на 2025 рік, враховуючи:
- погодні умови в Бразилії (використовуй openweathermap tool)
- попит на апельсини у світі (використовуй SerpAPI tool),
- економічну ситуацію (використовуй SerpAPI tool).

Видай мені наступний результат:
1. Знайди останні новини про погодні умови в Бразилії.
2. Знайди статті про глобальний попит на апельсини.
3. Оціни тренд експорту.
4. Зроби прогноз на 2025 (в тоннах) з поясненням.

Якщо не можеш знайти інформацію, то напиши, що не можеш знайти дані і не пробуй прогнозувати. Треба лише чітка оцінка.
"""

/Users/dmytro/Code/MachineLearning/machine-learning-for-humans-2.0/.env/lib/python3.12/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [24]:
agent_executor.invoke({'input': query})



> Entering new AgentExecutor chain...
I need to gather information on the weather conditions in Brazil, global demand for oranges, and the economic situation to make an informed estimate for orange exports in 2025. 

Action: Search
Action Input: "latest weather news Brazil October 2023"

Thought: I will first find the latest weather news in Brazil to understand the current conditions affecting orange production. 
[' entity_type: related_questions.', 'As per the latest update from Fundecitrus, there is a decline in the production of oranges for the São Paulo and West-Southwest Minas Gerais citrus belt in the ...', 'Fundecitrus and its cooperators released an orange crop forecast on Feb. 10. Total production is forecast to be 228.52 million boxes for 2024–25.', 'This update highlights the resilience of the Brazilian orange industry amid climatic challenges, with production forecasts remaining robust.', "Brazil's orange juice production will drop by almost 30% compared to the previous s

{'input': '\nМи експортуємо апельсини з Бразилії. В 2021 — 200т, в 2022 — 190т, в 2023 — 210т, в 2024 (ще не завершено) — 220т.\nЗроби оцінку на 2025 рік, враховуючи:\n- погодні умови в Бразилії (використовуй openweathermap tool)\n- попит на апельсини у світі (використовуй SerpAPI tool),\n- економічну ситуацію (використовуй SerpAPI tool).\n\nВидай мені наступний результат:\n1. Знайди останні новини про погодні умови в Бразилії.\n2. Знайди статті про глобальний попит на апельсини.\n3. Оціни тренд експорту.\n4. Зроби прогноз на 2025 (в тоннах) з поясненням.\n\nЯкщо не можеш знайти інформацію, то напиши, що не можеш знайти дані і не пробуй прогнозувати. Треба лише чітка оцінка.\n',
 'output': 'The estimated orange exports for 2025 are projected to be around 230-240 tons, considering the current weather conditions, global demand trends, and the economic situation in Brazil.'}

### Висновок

> Final Answer: The estimated orange exports for 2025 are projected to be around 230-240 tons, considering the current weather conditions, global demand trends, and the economic situation in Brazil.

Що із цього можна покращити - взагалі основне це покращити промпт, бо від нього багато чого залежить. Так наприклад openweathermap не був використав в процесі взагалі. Був використав пошук по погоді. Чомусь використовувся 2023 рік для аналізу, але тоді було б краще по кожному року із кожним експортом щоб зрозуміти що саме на то впливало. Також мабуть на одній все ж мові задавав опис tools та сам query - або все на українській, або на англійській, щоб більш точно було б.